# Google Play App's Review Data Scraper

> Aplikasi ini dibuat untuk melakukan penarikan data dari salah satu maupun beberapa aplikasi yang terdapat di Google Play Store ke dalam bentuk csv

Data yang diperoleh antara lain adalah :
- Review dari user
- Star Rating
- Hari dan Tanggal
- Informasi Aplikasi

## Setup

Let's install the required packages and setup the imports:

(Digunakan untuk melakukan instalasi package yang diperlukan dalam melakukan data scraping)

In [1]:
!pip install -qq google-play-scraper
!pip install watermark

You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
%reload_ext watermark
%watermark -v -p pandas,matplotlib,seaborn,google_play_scraper

Python implementation: CPython
Python version       : 3.10.0
IPython version      : 7.30.1

pandas             : not installed
matplotlib         : not installed
seaborn            : not installed
google_play_scraper: 1.0.2



In [3]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

ModuleNotFoundError: No module named 'pandas'

Aplikasi yang akan dilakukan data scraping, ditulis nama packagenya.
Dapat ditambahkan lebih dari 1 aplikasi

In [ ]:
app_packages = [
  'id.co.bitcoin',
]

## Scraping App Information

Let's scrape the info for each app:

In [ ]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='id', country='id')
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  3.99it/s]


Merubah data hasil scraping ke dalam bentuk JSON

In [ ]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

Data JSON dari aplikasi yang telah dilakukan scraping

In [ ]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 dan yang lebih tinggi",
  "appId": "id.co.bitcoin",
  "containsAds": false,
  "contentRating": "Rating 3+",
  "contentRatingDescription": null,
  "currency": "IDR",
  "description": "The biggest Indonesia's Crypto Assets marketplace within your hands!\r\n\r\nIndodax Official Mobile App!\r\n\r\nIndodax is the biggest Crypto Asset marketplace in Indonesia. We currently have more than 2 million members from Indonesia and all over the world. With the sophisticated technology, Indodax has improved its mobile application which eases all traders with a wide range of crypto assets.\r\n\r\nWith this we proudly present\r\n\r\n\u2605Buy and Sell Crypto Assets\r\n\r\nIndodax gives you the opportunity to buy Bitcoin and other crypto-assets with less an hour. We have the most liquid transaction on all the markets with 2 pairs which are Indonesian Rupiah and USDT.\r\nTo support your trading activity, we also accept vario

This contains lots of information including the number of ratings, number of reviews and number of ratings for each score (1 to 5). Let's ignore all of that and have a look at their beautiful icons:

In [ ]:
def format_title(title):
  sep_index = title.find(':') if title.find(':') != -1 else title.find('-')
  if sep_index != -1:
    title = title[:sep_index]
  return title[:10]

fig, axs = plt.subplots(2, len(app_infos) // 2, figsize=(14, 5))

for i, ax in enumerate(axs.flat):
  ai = app_infos[i]
  img = plt.imread(ai['icon'])
  ax.imshow(img)
  ax.set_title(format_title(ai['title']))
  ax.axis('off')

<Figure size 1008x360 with 0 Axes>

We'll store the app information for later by converting the JSON objects into a Pandas dataframe and saving the result into a CSV file:

In [ ]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

## Scraping App Reviews

In an ideal world, we would get all the reviews. But there are lots of them and we're scraping the data. That wouldn't be very polite. What should we do?

We want:

- Balanced dataset - roughly the same number of reviews for each score (1-5)
- A representative sample of the reviews for each app

We can satisfy the first requirement by using the scraping package option to filter the review score. For the second, we'll sort the reviews by their helpfulness, which are the reviews that Google Play thinks are most important. Just in case, we'll get a subset from the newest, too:

In [ ]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='id',
        country='id',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:05<00:00,  5.20s/it]


Note that we're adding the app id and sort order to each review. Here's an example for one:

In [ ]:
print_json(app_reviews[0])

{
  "appId": "id.co.bitcoin",
  "at": "2021-12-04 07:24:32",
  "content": "Indodax parah, buat kalian yang ingin terjun ke dunia crypto tolong jangan di apk indodax, setiap momen krusial indodax selalu error pas lagi dump error jadi gak bisa serok pas lagi pump juga error gak bias TP, tolong jangan kecewakan para pengguna, karena mereka trading menggunakan uang bukan daun. Indodax cuma cocok untuk dijadikan dompet saja buat transfer ke binance.",
  "repliedAt": "2021-12-04 10:11:05",
  "replyContent": "Dear member Indodax. Mohon maaf atas ketidaknyamanannya. Indodax akan terus berinovasi dan melakukan perbaikan layanan untuk kepuasan pengguna dalam bertransaksi. Apabila Anda mengalami kendala dalam bertransaksi melalui aplikasi ini, mohon langsung hubungi support@indodax.com, Live Chat atau di 021-50959800. Terima kasih.",
  "reviewCreatedVersion": "4.2.3",
  "reviewId": "gp:AOqpTOEWT5SQesX_avipO14GWH9mFTCFE6D2byHVhRHclHbYtbFYVdaNYf77bbz59EGU8biGq-Sc0z97frkaRDk",
  "score": 1,
  "sortO

`repliedAt` and `replyContent` contain the developer response to the review. Of course, they can be missing.

How many app reviews did we get?



In [ ]:
len(app_reviews)

1200

Let's save the reviews to a CSV file:

In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('Hasil Scraping.csv', index=None, header=True)

## Summary

Well done! You now have a dataset with more than 15k user reviews from 15 productivity apps. Of course, you can go crazy and get much much more.

- [Read the tutorial](https://www.curiousily.com/posts/create-dataset-for-sentiment-analysis-by-scraping-google-play-app-reviews-using-python/)
- [Run the notebook in your browser (Google Colab)](https://colab.research.google.com/drive/1GDJIpz7BXw55jl9wTOMQDool9m8DIOyp)
- [Read the `Getting Things Done with Pytorch` book](https://github.com/curiousily/Getting-Things-Done-with-Pytorch)

You learned how to:

- Set goals and expectations for your dataset
- Scrape Google Play app information
- Scrape user reviews for Google Play apps
- Save the dataset to CSV files

Next, we're going to use the reviews for sentiment analysis with BERT. But first, we'll have to do some text preprocessing!


## References

- [Google Play Scraper for Python](https://github.com/JoMingyu/google-play-scraper)